In [8]:
import os 
import pandas as pd 
import json 
import traceback

In [9]:
from langchain.chat_models import ChatOpenAI

In [10]:
from dotenv import load_dotenv

load_dotenv() # take environment variable from .env

True

In [11]:
key = os.getenv("OPENAI_API_KEY")

In [12]:
llm = ChatOpenAI(openai_api_key = key, model_name='gpt-3.5-turbo', temperature=0.5)

In [13]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7955e6508fd0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7955e650ada0>, temperature=0.5, openai_api_key='sk-proj-fErHxrfGpuyBYjT48YNST3BlbkFJc43sjaUOX096QPkvSEXf', openai_proxy='')

In [14]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate 
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [15]:
# response fromat 
RESPONSE_JSON={
    "1":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct" : "correct answer",
    },
    "2":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct" : "correct answer",
    },
    "3":{
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct" : "correct answer",
    },
}

In [16]:
prompt_template = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is you job to create a quiz of \
{number} multiple choice question for {subject} students in {tone} tone.
Make sure that the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to fromat your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [17]:
# prompt
mcq_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template= prompt_template
)

In [18]:
# by the chains we are connecting the our model with the prompt 
mcq_chain = LLMChain(llm = llm, prompt= mcq_generation_prompt, output_key="mcq", verbose=True)

In [19]:
# second template for evaluating the quiz
prompt_template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. only use at max 50 words for complexity. If the quiz is not as per the cognitive and analytical abilities of the students, \
update the quiz qestions which needs to be changed and change the tone such that it perfectly fits the student abilities.
Quiz_MCQs:
{mcq}

Check from an expert English Writer of the above quiz:
"""

In [20]:
mcq_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "mcq"],
    template = prompt_template2
)

In [21]:
review_chain= LLMChain(llm=llm,prompt=mcq_evaluation_prompt, output_key="review", verbose=True)

In [22]:
# by using the sequential chain we are connecting both the chains 
evaluation_chain= SequentialChain(chains=[mcq_chain, review_chain], input_variables=["text", "number", "subject", "tone","response_json"], output_variables=["mcq", "review"], verbose=True)

In [23]:
file_path="/home/umang.rathi/Documents/mcq_generator/data.txt"

In [24]:
# open and read the data from the file ehich is used for passing to the model for making quiz 
with open (file_path, 'r') as file:
    text=file.read()

In [25]:
print(text)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence. The synonym self-teaching computers was also used in this time period.

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes. In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells. Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data. Other researchers who have studied human cognitive systems contributed to the modern machine lear

In [26]:
# convert the python dictionary into a JSON fromatted string 
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [27]:
NUMBER = 5
SUBJECT = "Machine Learning"
tone = "simple"

In [29]:
# setup token usuage and tracking the cost using the callbacks 
with get_openai_callback() as cb:
    response = evaluation_chain(
        {
            "text" : text,
            "number" : NUMBER,
            "subject" : SUBJECT,
            "tone" : tone,
            "response_json" : json.dumps(RESPONSE_JSON)
        }
    )




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence. The synonym self-teaching computers was also used in this time period.

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes. In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells. Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communi

In [32]:
# print(cb)
print(f"Total tokens : {cb.total_tokens}")
print(f"Prompt Tokens : {cb.prompt_tokens}")
print(f"Completios Token : {cb.completion_tokens}")
print(f"Total Cost : {cb.total_cost}")

Total tokens : 1863
Prompt Tokens : 1394
Completios Token : 469
Total Cost : 0.003029


In [33]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence. The synonym self-teaching computers was also used in this time period.\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes. In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells. Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data. Other researchers who have studied human cognitive systems contributed to the modern

In [49]:
mcq = response.get('mcq')

In [50]:
# json.dumps -> serialize the pyhton object into a JSON formatted string 
# json.loads -> parse the json string and convert it into a python dictionary or list according to format.
mcq = json.loads(mcq)

In [51]:
mcq

{'1': {'mcq': "Who coined the term 'machine learning' in 1959?",
  'options': {'a': 'Donald Hebb',
   'b': 'Walter Pitts',
   'c': 'Arthur Samuel',
   'd': 'Raytheon Company'},
  'correct': 'c'},
 '2': {'mcq': 'What was the early machine learning model introduced by Arthur Samuel in the 1950s?',
  'options': {'a': 'Speech recognition',
   'b': 'Checkers game analysis',
   'c': 'Pattern recognition',
   'd': 'Image classification'},
  'correct': 'b'},
 '3': {'mcq': 'Who proposed the early mathematical models of neural networks to come up with algorithms that mirror human thought processes?',
  'options': {'a': 'Donald Hebb',
   'b': 'Warren McCulloch',
   'c': 'Tom M. Mitchell',
   'd': 'Alan Turing'},
  'correct': 'b'},
 '4': {'mcq': 'What is the main objective of modern-day machine learning?',
  'options': {'a': 'To generate random data',
   'b': 'To classify data based on developed models',
   'c': 'To predict lottery numbers',
   'd': 'To create new programming languages'},
  'corre

In [52]:
mcq_data = []
for key,value in mcq.items():
    mcq = value['mcq']
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value['options'].items()
        ]
    )
    correct = value['correct']
    mcq_data.append({"MCQ" : mcq, "Choices" : options, "Correct" : correct})

In [53]:
mcq_data

[{'MCQ': "Who coined the term 'machine learning' in 1959?",
  'Choices': 'a : Donald Hebb | b : Walter Pitts | c : Arthur Samuel | d : Raytheon Company',
  'Correct': 'c'},
 {'MCQ': 'What was the early machine learning model introduced by Arthur Samuel in the 1950s?',
  'Choices': 'a : Speech recognition | b : Checkers game analysis | c : Pattern recognition | d : Image classification',
  'Correct': 'b'},
 {'MCQ': 'Who proposed the early mathematical models of neural networks to come up with algorithms that mirror human thought processes?',
  'Choices': 'a : Donald Hebb | b : Warren McCulloch | c : Tom M. Mitchell | d : Alan Turing',
  'Correct': 'b'},
 {'MCQ': 'What is the main objective of modern-day machine learning?',
  'Choices': 'a : To generate random data | b : To classify data based on developed models | c : To predict lottery numbers | d : To create new programming languages',
  'Correct': 'b'},
 {'MCQ': 'According to Tom M. Mitchell, when is a computer program said to learn?

In [55]:
mcq = pd.DataFrame(mcq_data)

In [56]:
mcq

,MCQ,Choices,Correct
0,Who coined the term 'machine learning' in 1959?,a : Donald Hebb | b : Walter Pitts | c : Arthu...,c
1,What was the early machine learning model intr...,a : Speech recognition | b : Checkers game ana...,b
2,Who proposed the early mathematical models of ...,a : Donald Hebb | b : Warren McCulloch | c : T...,b
3,What is the main objective of modern-day machi...,a : To generate random data | b : To classify ...,b
4,"According to Tom M. Mitchell, when is a comput...",a : When it can think like a human | b : When ...,c


In [57]:
mcq.to_csv("machine_learning.csv", index=False)